In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models.tfidfmodel import TfidfModel

# Load the data :

In [2]:
reviews=pd.read_csv(r"C:\Users\mohz2\Desktop\Awooda\reviewly\data\yelp_review.csv")
business=pd.read_csv(r"C:/Users/mohz2/Desktop/Awooda/reviewly/data/yelp_business.csv")

In [3]:
reviews.head(5)

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
0,vkVSCC7xljjrAI4UGfnKEQ,bv2nCi5Qv5vroFiqKGopiw,AEx2SYEUJmTxVVB18LlCwA,5,2016-05-28,Super simple place but amazing nonetheless. It...,0,0,0
1,n6QzIUObkYshz4dz2QRJTw,bv2nCi5Qv5vroFiqKGopiw,VR6GpWIda3SfvPC-lg9H3w,5,2016-05-28,Small unassuming place that changes their menu...,0,0,0
2,MV3CcKScW05u5LVfF6ok0g,bv2nCi5Qv5vroFiqKGopiw,CKC0-MOWMqoeWf6s-szl8g,5,2016-05-28,Lester's is located in a beautiful neighborhoo...,0,0,0
3,IXvOzsEMYtiJI0CARmj77Q,bv2nCi5Qv5vroFiqKGopiw,ACFtxLv8pGrrxMm6EgjreA,4,2016-05-28,Love coming here. Yes the place always needs t...,0,0,0
4,L_9BTb55X0GDtThi6GlZ6w,bv2nCi5Qv5vroFiqKGopiw,s2I_Ni76bjJNK9yG60iD-Q,4,2016-05-28,Had their chocolate almond croissant and it wa...,0,0,0


# One restaurant :

In [4]:
mon_ami_gabi = reviews[reviews["business_id"]=="4JNXUYY8wbaaDmk3BPzlWw"]

In [5]:
mon_ami_gabi.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
1720,YCzBgTYCyTtPAe2v8_GGKA,CZs1Eq7t1fqpXV1yox-CVg,4JNXUYY8wbaaDmk3BPzlWw,4,2017-10-25,Traditional French fare at the heart of the La...,0,0,1
2399,lPf1wwTu9VeWABHDPhu2tA,Zqk-ye_OHeH_sS1JsdsLgg,4JNXUYY8wbaaDmk3BPzlWw,5,2013-10-15,"This is my go to place for the last 12 years, ...",4,0,2
3430,voVb586hGvZC6vUA7HUIIw,RmThjWcv7x5Z_vVuOlRNwQ,4JNXUYY8wbaaDmk3BPzlWw,2,2017-04-03,"Mediocre food for too much money, at least at ...",0,0,0
7921,jipTZe7L20v422d2Di2_Tw,hBl_w9Rmn_Bz0HiMzwaknA,4JNXUYY8wbaaDmk3BPzlWw,4,2012-12-26,My family and I had dinner here and the wait w...,0,0,0
8793,D5r-QLswHpnxaMadQdi_dg,eZ5I1bHkdXoGOFKn_oOFFQ,4JNXUYY8wbaaDmk3BPzlWw,5,2015-10-21,Patio dining at its best. Steak au poive with ...,0,0,0


In [6]:
mon_ami_gabi.count()

review_id      7362
user_id        7362
business_id    7362
stars          7362
date           7362
text           7362
useful         7362
funny          7362
cool           7362
dtype: int64

In [7]:
mon_ami_gabi_reviews = mon_ami_gabi[["text"]]

In [8]:
mon_ami_gabi_reviews.head(5)

,text
1720,Traditional French fare at the heart of the La...
2399,"This is my go to place for the last 12 years, ..."
3430,"Mediocre food for too much money, at least at ..."
7921,My family and I had dinner here and the wait w...
8793,Patio dining at its best. Steak au poive with ...


# Text processing ...


Tokenization + lowercasing + alphabetical words + no stop words + lemmatization:

In [18]:
tokens = []
wordnet_lemmatizer = WordNetLemmatizer()
for lab, row in mon_ami_gabi_reviews.iterrows():
    token = word_tokenize(row["text"].lower())
    alpha_only = [t for t in token if t.isalpha()]
    no_stops = [t for t in alpha_only if t not in stopwords.words('english')]
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
    tokens.append(lemmatized)
print(len(tokens))

7362


In [19]:
print(tokens[0])

['traditional', 'french', 'fare', 'heart', 'la', 'vega', 'strip', 'never', 'fails', 'satisfy', 'whether', 'regular', 'virgin', 'mon', 'ami', 'gabi', 'please', 'make', 'sure', 'save', 'room', 'dessert', 'perhaps', 'done', 'make', 'dining', 'experience', 'dessert', 'filled', 'table', 'made', 'scratch', 'sinfully', 'delicious', 'dessert', 'yes', 'sweet', 'seen', 'picture', 'dined', 'numerous', 'time', 'year', 'yes', 'food', 'dessert', 'onion', 'soup', 'steak', 'frites', 'mussel', 'mariniere', 'mussel', 'power', 'half', 'order', 'yes', 'half', 'clearly', 'utilize', 'doughy', 'stuff', 'scooping', 'white', 'wine', 'herb', 'broth', 'ca', 'go', 'wrong', 'pretty', 'much', 'anything', 'menu', 'offer', 'peruse', 'menu', 'order', 'away']


In [21]:
print(mon_ami_gabi_reviews.iloc[0]["text"])

Traditional French fare at the heart of the Las Vegas Strip that never fails to satisfy.  Whether you're a regular or virgin at Mon Ami Gabi please make sure you save room for dessert.  Perhaps do what I've done, just make the dining experience a dessert filled table;)  made from scratch and sinfully delicious desserts.  Yes, I have a sweet tooth- have you not seen my pictures!  
I've dined here numerous times over the years, yes Food and not only dessert.  
Onion soup, steak frites and those mussels mariniere! The mussels, I have will power and have the half order. Yes, half so I can clearly utilize that doughy stuff by scooping up the white wine & herbs broth:) 
You can't go wrong with pretty much anything the menu offers, so peruse that menu and order away!


# filter out restaurants:

In [9]:

# not properly filtered - for simplicity
restaurants=business[business['categories'] == 'Restaurants']

In [10]:
restaurants.head()

,business_id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
344,_--LAZXefssKWdyn-BTp9A,"""Fitch's Chophouse & Spirits""",NaN,"""5957 McKee Rd""",Fitchburg,WI,53719,43.014851,-89.457948,2.0,3,0,Restaurants
944,vZbcZDWqIvHmUtS3Pky1tA,"""Edo Japon""",Ville-Marie,"""705 Rue Sainte-Catherine O""",Montréal,QC,H3B 4G5,45.502755,-73.571472,4.5,7,1,Restaurants
1053,8b4amK9y5miP6-UzfH5K0A,"""Conviction""",NaN,"""609 King St W""",Toronto,ON,M5V 1MB,43.644221,-79.400655,3.5,3,0,Restaurants
1567,Gg0DVuNGaqUVFPVrrFQd6Q,"""Wing Wing Restaurant""",East York,"""3032 Danforth Ave""",East York,ON,M4C 1N2,43.690876,-79.289938,2.0,5,1,Restaurants
2711,BpU9-wcOWyKcVtkvnfRazA,"""Wok & Grill""",Downtown,"""120 5th Ave""",Pittsburgh,PA,15222,40.441888,-80.003443,3.0,3,1,Restaurants


In [11]:
restaurants.count()

business_id     362
name            362
neighborhood    102
address         362
city            362
state           362
postal_code     360
latitude        362
longitude       362
stars           362
review_count    362
is_open         362
categories      362
dtype: int64

# selecting restaurant reviews:

In [12]:
restaurant_reviews = reviews[reviews["business_id"].isin(restaurants["business_id"])]

In [13]:
restaurant_reviews.head()

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
574,sMlVBM9tF26Hbm1ufMJwGQ,dyhTHLIf6eWBvU78Y3T06A,HfApX8Ps1T-PpmwIVTo0iw,4,2008-05-12,"When I hit a quarter of a century in years, I ...",19,20,14
777,UAPtDhOg-S8scWmDbTa-Cw,5V8eXkTJb6IejJkMDaj_Bw,iAhv21whBCH7FjwCC2WyuQ,3,2009-10-07,Apparently Packer fans aren't Yelpers.. well t...,3,0,1
2251,vIAfE2dSKBrPfNx48Hp75A,qYxGJKlYrqNgodzMWHaaGw,I7yKjeRgOcGfk0JzPI8SXQ,4,2007-03-09,"This is a chain burger place, but they do make...",2,1,0
7781,H-4iwfPkxe5v5p2rtzUonQ,XuCbLgo9j1q5dDh9251vkg,AtOE7tSrJn1HXnU9YoUbPQ,2,2011-02-25,"When interviewing for a job, they say the inte...",2,2,2
11718,pGIl_9uxupp45yl8X_VIcw,oS0Z4spgZmf73OGw1YkLmA,DkaPGavPjjw9DWUmrZ19IQ,1,2015-06-02,"Let me sum it up: Food is horrible, mac and ch...",0,0,0


In [14]:
restaurant_reviews.count()

review_id      2312
user_id        2312
business_id    2312
stars          2312
date           2312
text           2312
useful         2312
funny          2312
cool           2312
dtype: int64

# users with most restaurant reviews:

In [15]:
user_agg=restaurant_reviews.groupby('user_id').agg({'review_id':['count']})

In [16]:
user_agg=user_agg.sort_values([('review_id','count')],ascending=False)

In [17]:
user_agg.head(10)

,review_id
,count
user_id,
PeLGa5vUR8_mcsn-fn42Jg,5
3Vd_ATdvvuVVgn_YCpz8fw,3
GRsPBPJ4Ckx4mpm2UbzWsg,3
sYQyXDjGaJj7wfaqz5u8KQ,3
Kf3prKlho75YWolB2CZXmg,3
HLaSqQMDVvlcFPGJL_kGCA,3
XEHZoTDWjw3w-gcQyYIe4g,3
2yLD3esBneRzDI4Uh26q-A,3


# top user reviews :

we'll call him user A.

In [18]:
user_A_reviews = restaurant_reviews[restaurant_reviews["user_id"] == "PeLGa5vUR8_mcsn-fn42Jg"]

In [19]:
user_A_reviews

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
2327543,EAP3SKIHPS5A8yCCk9r5wg,PeLGa5vUR8_mcsn-fn42Jg,qzh8srlptSOl2LaGWn3hDQ,3,2008-05-27,"Good food, good prices, and good service.\n\nN...",0,2,0
2327557,dggxkzx73GspVGWBeGBsHA,PeLGa5vUR8_mcsn-fn42Jg,6aImh7Tu2RS90AxCg3W3YA,2,2008-04-05,Nothing here to get excited about. Not sure if...,3,3,3
2500897,8bapkaVeCKFc9YjPWfSkeQ,PeLGa5vUR8_mcsn-fn42Jg,I-Fbhngu_uxPCCsJE8gpCA,2,2008-03-26,"The new ""Pischke's Blue Ribbion"" is a cleaner,...",2,0,1
2630819,B9HbZ915VwHk65EswfL4XQ,PeLGa5vUR8_mcsn-fn42Jg,c2x8wXMI4HbqBElLQ-tPRw,4,2008-04-30,One of the best Pretzels I have ever had (not ...,0,1,0
2767997,a3d6EtUzuhFGiQ4ucgMqqA,PeLGa5vUR8_mcsn-fn42Jg,HfApX8Ps1T-PpmwIVTo0iw,5,2008-03-24,"This is a tiny, independent establishment... a...",1,0,1


# Second top user reviews:  

we'll call him user B.

In [20]:
user_B_reviews = restaurant_reviews[restaurant_reviews["user_id"] == "3Vd_ATdvvuVVgn_YCpz8fw"]

In [21]:
user_B_reviews

,review_id,user_id,business_id,stars,date,text,useful,funny,cool
2416163,OSma_WqK1shK7fhmxM3OlQ,3Vd_ATdvvuVVgn_YCpz8fw,pAGbg3xqAvkHbasxKOAqjQ,3,2009-02-07,This was my 3rd time here. Another food trip i...,7,4,8
2494440,KMwhnsvMuxxiZOl2Ghf9tQ,3Vd_ATdvvuVVgn_YCpz8fw,lqEbAO2I1q4ttg2LvMStqQ,5,2009-09-17,F&#%$* FANTASTIC!!\n\nAnother FREEBIE for join...,9,3,7
2494475,crEy_wbWnMbn4cDPoCm5QQ,3Vd_ATdvvuVVgn_YCpz8fw,lD5lowsGjnU71NzA1L3-hA,1,2009-01-11,I had this place on my list but I didnt make i...,3,1,1


# Text processing user A's reviews:

In [22]:
user_A_dictionary = []
wordnet_lemmatizer = WordNetLemmatizer()
for lab, row in user_A_reviews.iterrows():
    token = word_tokenize(row["text"].lower())
    alpha_only = [t for t in token if t.isalpha()]
    no_stops = [t for t in alpha_only if t not in stopwords.words('english')]
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
    for t in lemmatized:
        user_A_dictionary.append(t)

In [23]:
user_A_dictionary

['good',
 'food',
 'good',
 'price',
 'good',
 'service',
 'nothing',
 'extreme',
 'extreme',
 'pita',
 'except',
 'maybe',
 'welcome',
 'alternative',
 'burger',
 'fry',
 'also',
 'generic',
 'decor',
 'inhibits',
 'digestion',
 'nothing',
 'get',
 'excited',
 'sure',
 'place',
 'changed',
 'hand',
 'recently',
 'remember',
 'lot',
 'people',
 'telling',
 'hot',
 'spot',
 'year',
 'ago',
 'anyway',
 'headed',
 'friday',
 'night',
 'year',
 'ago',
 'scene',
 'pathetic',
 'low',
 'life',
 'gangster',
 'vibe',
 'felt',
 'dirty',
 'overall',
 'crowd',
 'mostly',
 'punk',
 'service',
 'atrocious',
 'new',
 'pischke',
 'blue',
 'ribbion',
 'cleaner',
 'updated',
 'reinvented',
 'pischke',
 'really',
 'blow',
 'overpriced',
 'boring',
 'point',
 'keeping',
 'pischke',
 'name',
 'place',
 'whatsoever',
 'know',
 'tragedy',
 'struck',
 'place',
 'complete',
 'disembowelment',
 'pischke',
 'scottsdale',
 'ledgend',
 'chill',
 'place',
 'grab',
 'breakfast',
 'work',
 'ol',
 'hangover',
 'actual

In [24]:
user_A_bag_of_words = Counter(user_A_dictionary)

In [25]:
user_A_bag_of_words.most_common(10)

[('place', 5),
 ('good', 4),
 ('pischke', 4),
 ('service', 2),
 ('nothing', 2),
 ('extreme', 2),
 ('year', 2),
 ('ago', 2),
 ('headed', 2),
 ('felt', 2)]

# Text processing user B's reviews:

In [26]:
user_B_dictionary = []
wordnet_lemmatizer = WordNetLemmatizer()
for lab, row in user_B_reviews.iterrows():
    token = word_tokenize(row["text"].lower())
    alpha_only = [t for t in token if t.isalpha()]
    no_stops = [t for t in alpha_only if t not in stopwords.words('english')]
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in no_stops]
    for t in lemmatized:
        user_B_dictionary.append(t)

In [27]:
user_B_dictionary

['time',
 'another',
 'food',
 'trip',
 'inspired',
 'gift',
 'certificate',
 'arrived',
 'friday',
 'night',
 'parking',
 'lot',
 'fairly',
 'full',
 'entered',
 'dining',
 'room',
 'bar',
 'asked',
 'sit',
 'upstairs',
 'game',
 'room',
 'empty',
 'thought',
 'pool',
 'table',
 'fooseball',
 'pinball',
 'racing',
 'hunting',
 'jukebox',
 'plus',
 'big',
 'screen',
 'couch',
 'lounge',
 'projection',
 'screen',
 'event',
 'etc',
 'also',
 'small',
 'bar',
 'upstairs',
 'waitress',
 'run',
 'stair',
 'soft',
 'drink',
 'food',
 'ordered',
 'drink',
 'cheeseburger',
 'slider',
 'appetizer',
 'ordered',
 'sandwich',
 'steak',
 'son',
 'got',
 'teriyaki',
 'swiss',
 'burger',
 'fry',
 'wife',
 'got',
 'turkey',
 'bacon',
 'club',
 'got',
 'mustang',
 'new',
 'york',
 'bleu',
 'cheese',
 'bacon',
 'bit',
 'food',
 'arrived',
 'group',
 'kid',
 'parent',
 'arrived',
 'add',
 'life',
 'place',
 'kid',
 'excitedly',
 'enjoying',
 'game',
 'service',
 'great',
 'food',
 'however',
 'hamburger'

In [28]:
user_B_bag_of_words = Counter(user_B_dictionary)

In [29]:
user_B_bag_of_words.most_common(10)

[('burger', 5),
 ('food', 4),
 ('steak', 4),
 ('place', 4),
 ('another', 3),
 ('arrived', 3),
 ('bar', 3),
 ('got', 3),
 ('bacon', 3),
 ('bleu', 3)]

# Tf-idf:

In [38]:
# first we create one doc to contain the tokens of both users:
tokenized_docs = [user_A_dictionary,user_B_dictionary]

In [39]:
len(tokenized_docs)

2

In [40]:
tokenized_docs

[['good',
  'food',
  'good',
  'price',
  'good',
  'service',
  'nothing',
  'extreme',
  'extreme',
  'pita',
  'except',
  'maybe',
  'welcome',
  'alternative',
  'burger',
  'fry',
  'also',
  'generic',
  'decor',
  'inhibits',
  'digestion',
  'nothing',
  'get',
  'excited',
  'sure',
  'place',
  'changed',
  'hand',
  'recently',
  'remember',
  'lot',
  'people',
  'telling',
  'hot',
  'spot',
  'year',
  'ago',
  'anyway',
  'headed',
  'friday',
  'night',
  'year',
  'ago',
  'scene',
  'pathetic',
  'low',
  'life',
  'gangster',
  'vibe',
  'felt',
  'dirty',
  'overall',
  'crowd',
  'mostly',
  'punk',
  'service',
  'atrocious',
  'new',
  'pischke',
  'blue',
  'ribbion',
  'cleaner',
  'updated',
  'reinvented',
  'pischke',
  'really',
  'blow',
  'overpriced',
  'boring',
  'point',
  'keeping',
  'pischke',
  'name',
  'place',
  'whatsoever',
  'know',
  'tragedy',
  'struck',
  'place',
  'complete',
  'disembowelment',
  'pischke',
  'scottsdale',
  'ledgen

In [41]:
# Now we create a dictionary - mapping with ID for each token :
dictionary = Dictionary(tokenized_docs)

In [42]:
dictionary.token2id

{'actually': 0,
 'ago': 1,
 'also': 2,
 'alternative': 3,
 'anyway': 4,
 'appetizer': 5,
 'atrocious': 6,
 'bad': 7,
 'best': 8,
 'blow': 9,
 'blue': 10,
 'boring': 11,
 'breakfast': 12,
 'burger': 13,
 'chain': 14,
 'changed': 15,
 'chill': 16,
 'cleaner': 17,
 'complete': 18,
 'course': 19,
 'crowd': 20,
 'damn': 21,
 'decor': 22,
 'dessert': 23,
 'digestion': 24,
 'dirty': 25,
 'disembowelment': 26,
 'dog': 27,
 'eating': 28,
 'enjoyed': 29,
 'establishment': 30,
 'ever': 31,
 'everything': 32,
 'except': 33,
 'excited': 34,
 'extreme': 35,
 'felt': 36,
 'food': 37,
 'foodie': 38,
 'friday': 39,
 'friend': 40,
 'fry': 41,
 'gangster': 42,
 'generic': 43,
 'get': 44,
 'good': 45,
 'grab': 46,
 'hand': 47,
 'hangover': 48,
 'happened': 49,
 'headed': 50,
 'highly': 51,
 'home': 52,
 'hot': 53,
 'http': 54,
 'independent': 55,
 'inhibits': 56,
 'keeping': 57,
 'kind': 58,
 'know': 59,
 'ledgend': 60,
 'life': 61,
 'like': 62,
 'lot': 63,
 'loved': 64,
 'low': 65,
 'main': 66,
 'make': 

In [43]:
# Using the dictionary we can then create a corpus :
corpus = [dictionary.doc2bow(doc) for doc in tokenized_docs]

In [45]:
# Each tuple contains a token id and frequency of each token in the document.
print(corpus)

[[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 2), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 4), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 4), (84, 1), (85, 5), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1)

In [47]:
# Then we build our Tf-idf model: here each tuple contain token id and weight.
tfidf = TfidfModel(corpus)

In [48]:
print(tfidf)

TfidfModel(num_docs=2, num_nnz=291)


In [53]:
# Now we will create a similarity measure object in tf-idf space:
sims = gensim.similarities.Similarity(r'C:\Users\mohz2\Desktop\Awooda\reviewly',tfidf[corpus],num_features=len(dictionary))

In [54]:
print(sims)
print(type(sims))

Similarity index with 2 documents in 0 shards (stored under C:\Users\mohz2\Desktop\Awooda\reviewly)
<class 'gensim.similarities.docsim.Similarity'>


In [55]:
# lets consider user_A_dictionary as query document :
user_A_bow = dictionary.doc2bow(user_A_dictionary)

In [56]:
print(user_A_bow)

[(0, 1), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 2), (32, 1), (33, 1), (34, 1), (35, 2), (36, 2), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 4), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 4), (84, 1), (85, 5), (86, 1), (87, 1), (88, 2), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 2), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 1),

In [57]:
# use the tfidf model to find the weights of tokens in user_A_bow:
user_A_tfidf = tfidf[user_A_bow]

In [58]:
print(user_A_tfidf)

[(0, 0.08362420100070908), (1, 0.16724840200141816), (3, 0.08362420100070908), (4, 0.08362420100070908), (6, 0.08362420100070908), (7, 0.08362420100070908), (9, 0.08362420100070908), (10, 0.08362420100070908), (11, 0.08362420100070908), (12, 0.08362420100070908), (14, 0.08362420100070908), (15, 0.08362420100070908), (16, 0.08362420100070908), (17, 0.08362420100070908), (18, 0.08362420100070908), (19, 0.08362420100070908), (20, 0.08362420100070908), (22, 0.08362420100070908), (23, 0.08362420100070908), (24, 0.08362420100070908), (25, 0.08362420100070908), (26, 0.08362420100070908), (27, 0.08362420100070908), (28, 0.08362420100070908), (29, 0.08362420100070908), (30, 0.08362420100070908), (32, 0.08362420100070908), (33, 0.08362420100070908), (34, 0.08362420100070908), (35, 0.16724840200141816), (36, 0.16724840200141816), (38, 0.08362420100070908), (40, 0.08362420100070908), (42, 0.08362420100070908), (43, 0.08362420100070908), (44, 0.08362420100070908), (46, 0.08362420100070908), (47, 0.

# Computing similarity:

In [60]:
# Now we compare user_A_dictionary with the two doc in our tfidf model i.e user_A_dictionary & user_B_dictionary:
sims[user_A_tfidf]

array([0.99999994, 0.        ], dtype=float32)

Similarity between user_A_dictionary & user_A_dictionary is ~ 1. 